In [35]:
import pyspark
import re
import numpy

In [2]:
spark_context = pyspark.SparkContext("local", "exercise-2")

25/03/26 16:22:40 WARN Utils: Your hostname, jovan-laptop resolves to a loopback address: 127.0.1.1; using 192.168.8.175 instead (on interface wlp3s0)
25/03/26 16:22:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/26 16:22:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [46]:
from numpy import uint32
from numpy import uint64
import random

def generate_random_hash_params_A_P():
    A = random.getrandbits(64)
    P = random.getrandbits(64)
    while A>=P:
        A = random.getrandbits(64)
        P = random.getrandbits(64)
    return A , P
    
Ashingle, Pshingle = generate_random_hash_params_A_P()

def string_hash(shingle,slen,A,P):
    tmp = uint64(ord(shingle[0]))
    for i in range(1, slen):
        tmp = (tmp*uint64(A) + uint64(ord(shingle[i]))) % uint64(P)
    return uint32(tmp&uint64(0xFFFFFFFF))

In [47]:
def shingle(text: str, shingle_length: int):
    text = text.lower()
    # Remove all characters from 'text' that are not lowercase letters (a-z).
    # This includes spaces, numbers, punctuation, and uppercase letters.
    text = re.sub(r'[^a-z]', '', text)

    for i in range(len(text) - shingle_length + 1):
        yield string_hash(text[i:i+shingle_length], shingle_length, Ashingle, Pshingle)

In [48]:
def extract_filename(full_directory):
    filename = full_directory.split('/')[-1]
    return filename

In [49]:
rdd = spark_context.wholeTextFiles('./data/*.txt')
for filename, content in rdd.take(3):
    print(f"file: {filename}")
    print(f"content preview: {content[:200]}...\n") # show first 200 characters


file: file:/home/jovan/repos/data.ml.370/exercise-2/data/bank.txt
content preview: A bank is a financial intermediary and money creator that creates money by lending money to a borrower, thereby creating a corresponding deposit on the bank's balance sheet. Lending activities can be ...

file: file:/home/jovan/repos/data.ml.370/exercise-2/data/cat.txt
content preview: This article is about the cat species that is commonly kept as a pet. For the cat family, see Felidae. For other uses, see Cat (disambiguation) and Cats (disambiguation).
For technical reasons, "Cat ...

file: file:/home/jovan/repos/data.ml.370/exercise-2/data/catfood.txt
content preview: Cat food
For the song by King Crimson, see Cat Food (song).
Cat with a bowl of pelleted cat food.

Cat food is food intended for consumption by cats. Cats have requirements for their specific diet...



In [54]:
shingle_length = 12
files_shingles = rdd.map(lambda p: (extract_filename(p[0]), list(shingle(p[1], shingle_length))))

In [55]:
results = files_shingles.collect()
print(files_shingles.take(1))
# print(results[0])

/tmp/ipykernel_7462/2818458014.py:18: RuntimeWarning: overflow encountered in scalar multiply
/tmp/ipykernel_7462/2818458014.py:18: RuntimeWarning: overflow encountered in scalar multiply


[('bank.txt', [np.uint32(988559312), np.uint32(112026521), np.uint32(1081472970), np.uint32(2310261146), np.uint32(3070557045), np.uint32(80858866), np.uint32(3513334986), np.uint32(263077669), np.uint32(1396561804), np.uint32(4078663387), np.uint32(1783753748), np.uint32(2198632783), np.uint32(3944069899), np.uint32(2940712161), np.uint32(485758903), np.uint32(2920466251), np.uint32(3765760432), np.uint32(2677305901), np.uint32(3966488789), np.uint32(3442868345), np.uint32(1055574277), np.uint32(3728810837), np.uint32(1298189806), np.uint32(3584208923), np.uint32(1594786855), np.uint32(2677455364), np.uint32(1553995986), np.uint32(3851129787), np.uint32(3830043450), np.uint32(1051177430), np.uint32(513323621), np.uint32(1716621838), np.uint32(1736498488), np.uint32(816084843), np.uint32(3129156800), np.uint32(190590863), np.uint32(3269456126), np.uint32(1969052744), np.uint32(3303070591), np.uint32(3393176094), np.uint32(2091530022), np.uint32(3672156753), np.uint32(1015534066), np.ui

In [56]:
def uint32_hash(stringhashes,A):
    for i in range(0, len(stringhashes)):
        stringhashes[i] = uint32((uint64(A)*uint64(stringhashes[i])) >> uint64(32))
    return stringhashes

def min_hash(hashes,minhash_A):
    minhashes = []
    for i in range(0,len(minhash_A)):
        minhashes.append(min(uint32_hash(hashes,minhash_A[i])))
    return minhashes



In [57]:
num_minhashes = 50
minhash_A = []
for i in range(0,num_minhashes):
    minhash_A.append( random.getrandbits(64) )

minhashes = files_shingles.map(lambda p: (p[0],min_hash(p[1],minhash_A)))
minhashes.cache() #cache result for performance
res = minhashes.collect()

for x in res:
    jaccrow = minhashes.map(lambda p: (p[0],sum(numpy.array(p[1])- numpy.array(x[1])==0)/num_minhashes)).collect()
    for y in jaccrow:
        print(x[0].ljust(20) + " vs. " + y[0].ljust(20) + " similarity: " + str(y[1]*100) + " %")

/tmp/ipykernel_7462/2818458014.py:18: RuntimeWarning: overflow encountered in scalar multiply
/tmp/ipykernel_7462/4143354612.py:3: RuntimeWarning: overflow encountered in scalar multiply


bank.txt             vs. bank.txt             similarity: 100.0 %
bank.txt             vs. cat.txt              similarity: 0.0 %
bank.txt             vs. catfood.txt          similarity: 0.0 %
bank.txt             vs. hadoop.txt           similarity: 0.0 %
bank.txt             vs. hares.txt            similarity: 0.0 %
bank.txt             vs. lsh.txt              similarity: 0.0 %
bank.txt             vs. mapreduce.txt        similarity: 0.0 %
bank.txt             vs. minhash.txt          similarity: 0.0 %
bank.txt             vs. rabbit.txt           similarity: 0.0 %
bank.txt             vs. setsim.txt           similarity: 0.0 %
bank.txt             vs. swissbank.txt        similarity: 0.0 %
bank.txt             vs. tortoise.txt         similarity: 0.0 %
bank.txt             vs. turtles.txt          similarity: 0.0 %
bank.txt             vs. universal_hash.txt   similarity: 0.0 %
cat.txt              vs. bank.txt             similarity: 0.0 %
cat.txt              vs. cat.txt      